In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib as mpl 
import matplotlib.pyplot as plt

### deseq OUT

In [4]:
# podejcie MW #
#--------------#
# wyniki deseq:
# pvalue jest normalizowane do padj przez "the Holm method" http://www.math.ucla.edu/~anderson/rw1001/library/base/html/p.adjust.html
# w analizie odciecie jest przy padj = 0.001 (mozliwe bylo w DEBROWSER) i taki mam plik wynikowy, podzielony na UP/DOWN
# (w zasadzie dwa pliki), z ktorych na podstawie najwyzszego log2foldchange wzialem po 100 najwyzszch sekwencji,
# polaczylem razem i zblastowalem. Plik ktory jest ponizej to zlepek 100UP i 100DOWN wynikow deseq, tych samych, ktore sie blastowaly

In [3]:
df_deseq_out = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/1d/PBS1d_vs_MC1d_DESEQ_100UP_100DOWN_together.csv', sep='\t')
#df_deseq_out = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/PBS1d_vs_MC6d_DESEQ_100UP_100DOWN_together.csv')
#df_deseq_out = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/PBS1d_vs_MC9d_DESEQ_100UP_100DOWN_together.csv')
df_deseq_out.head()

,ID,PBS_1d_1,PBS_1d_2,PBS_1d_3,PBS_1d_4,MC_1d_1,MC_1d_2,MC_1d_3,padj,log2FoldChange,pvalue,foldChange,log10padj
0,TRINITY_DN96009_c1_g3_i3,0.0,0.0,0.0,0.000000,1926.746939,838.569902,912.024085,1.819644e-43,12.791799,2.678902e-47,7091.127494,42.740014
1,TRINITY_DN96009_c1_g7_i1,0.0,0.0,0.0,0.000000,1545.529985,939.097763,954.080531,2.081684e-60,12.695019,1.094528e-64,6631.036435,59.681585
2,TRINITY_DN96413_c4_g3_i1,0.0,0.0,0.0,0.000000,1395.729284,899.724351,217.491910,4.846018e-22,12.242750,5.707493e-25,4846.575702,21.314615
3,TRINITY_DN80414_c8_g1_i6,0.0,0.0,0.0,0.000000,373.985197,226.187686,1210.024049,2.914640e-20,11.768755,4.260318e-23,3489.378944,19.535415
4,TRINITY_DN91051_c3_g1_i1,0.0,0.0,0.0,0.905415,1429.821857,965.067460,561.153159,7.112150e-42,11.755678,1.271429e-45,3457.893280,41.147999


# filtrowanie pliku wynikowego blasta

In [3]:
df_blast_results = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/blast_salmo_ony.csv', sep='\t', low_memory=False)
df_blast_results.shape

(5265, 5)

### odfiltrowanie zbyt niskiego coverage (pomijamy w tej wersji salmo only)

### filtrowanie mRNA only

In [4]:
# select best hit based on evalue (drop_duplicates)
df_best_hit = df_blast_results.sort_values('evalue', ascending=True).drop_duplicates('query').sort_index()


In [5]:
df_best_hit.shape

(146, 5)

In [6]:
# select what you need based on string

# w tym wypadku mRNA only oznacza pominiecie blastów z ncRNA, mixRNA i innych dziwnych, nie oznacza natomiast
# zabawy z chromosome i filtrowaniem tego dodatkowo, poniewaz blastujac tylko do salmo salar, nie mam (chyba)
# zadnych hitow chromosome. Chromosome (czyli hity bez mRNA) w tym wypadku to takie, ktore w blascie mialy NO HIT.
# czyli zamiast filtrowania po chromosome nalezy wrzucic w te grupe trinity, ktore mialy NO HIT.

df_mRNA_only = df_best_hit[df_best_hit['al_title'].str.contains('mRNA')]
df_mRNA_only.shape


(140, 5)

In [7]:
df_mRNA_only.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/PBS1d_vs_MCLR9d_mRNA_only_wo_cov20.csv', index=False)

In [8]:
# łączenie odfiltrowanych mRNA z wynikami deseq

df_mRNA_only.rename(columns={'query' : 'ID'}, inplace=True)
df_mRNA_only.head()

/home/maciek/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,ID,al_title,al_accession,evalue,coverage
0,TRINITY_DN103835_c12_g1_i10,gi|929125660|ref|XM_014125024.1| PREDICTED: Sa...,XM_014125024,5.793890e-13,19.583333
1,TRINITY_DN106993_c0_g2_i1,gi|929283969|ref|XM_014163494.1| PREDICTED: Sa...,XM_014163494,1.500890e-63,84.444444
2,TRINITY_DN95065_c7_g1_i9,gi|929231010|ref|XM_014136583.1| PREDICTED: Sa...,XM_014136583,2.025710e-32,12.116564
104,TRINITY_DN90972_c8_g1_i1,gi|929280255|ref|XM_014161528.1| PREDICTED: Sa...,XM_014161528,3.389730e-30,7.692308
210,TRINITY_DN96539_c2_g1_i1,gi|929304890|ref|XM_014174496.1| PREDICTED: Sa...,XM_014174496,4.620110e-173,40.988593


In [9]:
result = pd.merge(df_deseq_out, df_mRNA_only, on='ID')
result.sort_values(by='log2FoldChange', inplace=True)

In [10]:
result.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/mRNA_PBS_1d_vs_MC2_9d_deseq_w_mRNA_names.csv', index=False)

# TO DO: XM z tego ^^ pozamieniac na XP w unirefie

XM zamienilem na XP, ale tez na Entry, czyli taki akcession, ktory ma w sobie wszystkie XM danego rodzaju, czyli wszystkie transcript warianty mieszcza sie pod tym numerem. dlatego bede porownywal ze soba nie XP, ale wlasnie Entry 

In [4]:
mRNA = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/1d/salmo_salar_only/FINAL_RESULT/mRNA_PBS_1d_vs_MC2_1d_deseq_w_mRNA_names.csv')
#mRNA = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/salmo_salar_only/FINAL_RESULT/mRNA_PBS_1d_vs_MC2_6d_deseq_w_mRNA_names.csv')
#mRNA = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/mRNA_PBS_1d_vs_MC2_9d_deseq_w_mRNA_names.csv')

In [5]:
mRNA.head()

,ID,PBS_1d_1,PBS_1d_2,PBS_1d_3,PBS_1d_4,MC_6d_1,MC_6d_2,MC_6d_3,padj,log2FoldChange,pvalue,foldChange,log10padj,al_title,al_accession,evalue,coverage
0,TRINITY_DN99250_c2_g1_i11,133.321807,61.189811,85.545861,388.751130,0.000000,0.000000,0.0,3.221348e-12,"-10,0036064708546",1.583056e-13,0.000974,11.491962,gi|929103804|ref|XM_014207544.1| PREDICTED: Sa...,XM_014207544,1.315190e-06,9.565217
1,TRINITY_DN95730_c6_g1_i1,336.014309,145.451191,154.640595,47.602179,0.000000,0.000000,0.0,1.463072e-13,"-10,0346168100071",6.035825e-15,0.000953,12.834734,gi|929335812|ref|XM_014185888.1| PREDICTED: Sa...,XM_014185888,3.579930e-22,11.173975
2,TRINITY_DN111070_c5_g1_i1,401.049337,787.442653,610.885186,788.411093,0.000000,1.835489,0.0,5.516161e-40,"-10,0946395263019",1.478208e-42,0.000915,39.258363,gi|929240879|ref|XM_014140918.1| PREDICTED: Sa...,XM_014140918,1.596360e-50,23.388582
3,TRINITY_DN108512_c0_g5_i1,557.133403,803.492440,809.395453,410.568795,1.679643,0.000000,0.0,4.176010e-39,"-10,1005511170429",1.220047e-41,0.000911,38.379238,gi|929143564|ref|XM_014127772.1| PREDICTED: Sa...,XM_014127772,9.607620e-48,28.832117
4,TRINITY_DN108989_c16_g1_i1,114.895215,153.476084,257.734324,198.342413,0.000000,0.000000,0.0,1.181058e-25,"-10,1184268669784",1.232673e-27,0.000900,24.927729,gi|929107150|ref|XM_014209300.1| PREDICTED: Sa...,XM_014209300,5.782300e-45,20.299728


In [5]:
mRNA.shape

(117, 17)

krok 1: dodac do tabelki mRNA deseq 'Entry' (de facto wszystko z tej tabelki, ale tylko Entry mnie interesuje najbardziej) na podstawie 'al_accesion'

In [9]:
uniprot_converted = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/1d/salmo_salar_only/FINAL_RESULT/1d_XM_to_XP_Entry.tab', sep='\t')
#uniprot_converted = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/salmo_salar_only/FINAL_RESULT/6d_XM_to_XP_Entry.tab', sep='\t')
#uniprot_converted = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/9d_XM_to_XP_Entry.tab', sep='\t')

In [10]:
uniprot_converted.head(1)

,al_accession,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Cross-reference (RefSeq),Gene ontology (biological process),Gene ontology (cellular component),Gene ontology (GO),Gene ontology (molecular function),Gene ontology IDs
0,XM_014155425,A0A1S3N679,A0A1S3N679_SALSA,unreviewed,leucine-rich repeat and calponin homology doma...,LOC106577445,Salmo salar (Atlantic salmon),222,XP_014010900.1;XP_014010901.1;XP_014010902.1;,NaN,NaN,NaN,NaN,NaN


In [11]:
uniprot_converted.shape

(111, 14)

In [15]:
uniprot_converted_entry = uniprot_converted[['al_accession', 'Entry']]

In [20]:
# uniprot_converted_entry.shape

(120, 2)

In [12]:
merged = pd.merge(mRNA, uniprot_converted, on='al_accession') 

In [13]:
merged.shape # BT odpadly, nie dalo sie ich przeksztalcic na XP, na razie zostawiam, ale trzeba bedzie je zamienic na XM

(116, 30)

In [14]:
merged.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/1d/salmo_salar_only/FINAL_RESULT/merged.csv', index=False)
#merged.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/salmo_salar_only/FINAL_RESULT/merged.csv', index=False)
#merged.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/merged.csv', index=False)

krok 2: na podsawie 'Entry' polaczyc ^^ powyzsza tabelke z tabela bialek

In [15]:
deseq_proteins = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/proteins_salmo/PBS1d_MC1d_up+down_DESEQ_prawdopodobnie_wszystkie_with_protein_names.csv', low_memory=False)
#deseq_proteins = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/proteins_salmo/PBS1d_MC6d_up+down_DESEQ_prawdopodobnie_wszystkie_with_protein_names.csv', low_memory=False)
#deseq_proteins = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/proteins_salmo/PBS1d_MC9d_up+down_DESEQ_prawdopodobnie_wszystkie_with_protein_names.csv', low_memory=False)

In [16]:
merged_with_proteins_deseq = pd.merge(merged, deseq_proteins, on='Entry', suffixes=('_UTR', '_PROTEIN'))

In [17]:
merged_with_proteins_deseq.shape

(219, 57)

In [18]:
merged_with_proteins_deseq.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/1d/salmo_salar_only/FINAL_RESULT/mRNA_UTR_PROTEINS_DESEQ_merged_1d.csv', index=False)
#merged_with_proteins_deseq.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/salmo_salar_only/FINAL_RESULT/mRNA_UTR_PROTEINS_DESEQ_merged_6d.csv', index=False)
#merged_with_proteins_deseq.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/mRNA_UTR_PROTEINS_DESEQ_merged_9d.csv', index=False)

### filtrowanie hitów innych niz mRNA: ncRNA, miscRNA. ZDECYDOWAC CO Z NIMI

In [28]:
non_mRNA = df_best_hit[~df_best_hit['al_title'].str.contains('mRNA')]


In [29]:

non_mRNA.shape

(6, 5)

In [30]:
non_mRNA.head(1)

,query,al_title,al_accession,evalue,coverage
1045,TRINITY_DN106385_c5_g2_i1,gi|929246779|ref|XR_001320874.1| PREDICTED: Sa...,XR_001320874,4.447370e-19,15.862069


In [31]:
non_mRNA.rename(columns={'query' : 'ID'}, inplace=True)

In [32]:
result2 = pd.merge(df_deseq_out, non_mRNA, on='ID')
result2.sort_values(by='log2FoldChange', inplace=True)

In [33]:
result2.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/non_mRNA_PBS_1d_vs_MC2_9d_deseq_w_mRNA_names.csv', index=False)

### filtrowanie NO HIT
#### chromosome poprzednio == no hit teraz

In [35]:
# wszystkie blastowane trinity (100UP + 100DOWN)
all_trinity_to_blast = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/100_UP_100_DOWN_best_fold_change.headers')

In [36]:
all_trinity_to_blast.shape

(200, 1)

In [37]:
df_best_hit.rename(columns={'query' : 'ID'}, inplace=True)

In [38]:
# wszystkie odjac te ktore mialy hit w blascie, jakikolwiek hit
no_hit = all_trinity_to_blast[~all_trinity_to_blast['id'].isin(df_best_hit['ID'])]

In [39]:
no_hit.shape

(54, 1)

In [40]:
no_hit.rename(columns={'id' : 'ID'}, inplace=True)

/home/maciek/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [41]:
# lista NO HIT czyli dawna chromosomes, ktora idzie do analizy novel lncRNA
result3 = pd.merge(df_deseq_out, no_hit, on='ID')
result3.sort_values(by='log2FoldChange', inplace=True)

In [42]:
result3.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/9d/salmo_salar_only/FINAL_RESULT/NO_HIT_PBS_1d_vs_MC2_9d_deseq_w_mRNA_names.csv', index=False)

In [16]:
wszystkie_test = 51 + 17 + 132
wszystkie_test

200